In [1]:
import requests
import json

from pathlib import Path
from pprint import pprint
from datetime import datetime

from openai import OpenAI
from langchain_openai import OpenAI, ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains import LLMChain

from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


In [2]:
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()
openai_api_key = os.environ.get("OPENAI_API_KEY")

## Prompt
- load prompt
- load template with prompt

In [130]:
with open("prompt_continuouseffects.txt", "r") as file:
    prompt_continuous_effects = file.read()
print(prompt_continuous_effects)

prompt_template = PromptTemplate.from_template(prompt_continuous_effects)
#prompt_template = ChatPromptTemplate.from_template(prompt_continuous_effects)

"""
You will get two cards from the cardgame Magic: the Gathering. {card1} and {card2} both have continuous effects.
Use the {rules_db} as context for your decision.
You will get a {question} about two cards and give a judging regarding two contradicting continuous effects.
Make sure your judging, explanation und quoting of the rules don't contradict each other. If you make a decision for one rule, you have to stick with it.

You will follow these steps:

1. Give the oracle texts from both cards.
2. Decide which rule is fitting in the case: Timestamp (613.1 to 613.7) or Dependency (613.8).
2a. If you choose Timestamp, you can assume that {card1} is played first and has the earlier timestamp, while {card2} is played second and has the later timestamp.
2b. If you choose Timestamp, determine the layer on which the effects of the cards are applied. Layers are defined in 613.1 in the {rules_db}.
3. Quote the rule you think would apply to the given case of {card1} and {card2}.
4. Explain why

## RAG
= Retrieved Augmented Generation
- ChatGPT's knowlegde of the rules is from January 2022: for current state the rulebook needs to be retrieved and passed to the LLM
- this data can provide context for the specific questions
### Loader
- simple loader to load .txt data: complete rule book from Magic: The Gathering

In [71]:
#loader = TextLoader("./data/rules_shortened.txt")
loader = TextLoader("./data/rules.txt")
rules_doc = loader.load()

### Splitter
- splitting the text into smaller chunks
- this text splitter splits text recursively by characters: ["\n\n", "\n", " ", ""], so first paragraphs, then sentences, then words
- keeps semantically related pieces together as long as possible

In [72]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
rules_splits = text_splitter.split_documents(rules_doc)
# print(rules_splits[0])

### Vector Store
- for storage of the embeddings
- embedding for capture semantic meaning of the text: that's how context is given for later question
- OpenAI embedding for ChatGPT
- Chroma as open-source embedding database
- finally, construct a retriever out of vectorstore 

In [73]:
rules_db = Chroma.from_documents(documents=rules_splits, embedding=OpenAIEmbeddings())
retriever_rules = rules_db.as_retriever()

- create new LLM: temperature 0 for more consistent, non-creative and therefor comparable output

In [74]:
# llm = OpenAI(temperature=0, openai_api_key=openai_api_key)
model = ChatOpenAI(temperature=0, openai_api_key=openai_api_key)

## Case 1: Blood Moon vs. Urborg, Tomb of Yawgmoth
- Get card information via scryfall.com and delete unnecessary data: 
- keep only: name, mana cost, converted mana cost, type, oracle text, colors, keywords

In [131]:
card1 = "Urborg, Tomb of Yawgmoth"
response = requests.get(f'https://api.scryfall.com/cards/named?exact={card1} ', 
                 headers={'Accept': 'application/json'})
if response.status_code == 200:
    # Parse the JSON response into a Python dictionary to delete keywords
    data = response.json()
    keywords = ["object", "id", "oracle_id", "mtgo_foil_id", "rarity", "color_identity", "penny_rank", "multiverse_ids", "mtgo_id", "tcgplayer_id", "cardmarket_id", "lang", "released_at", "uri", "scryfall_uri", "layout", "frame_effects", "highres_image", "image_status", "image_uris", "legalities", "games", "reserved", "foil", "nonfoil", "finishes", "oversized", "promo", "reprint", "variation", "set_id", "set", "set_name", "set_type", "set_uri", "set_search_uri", "scryfall_set_uri", "rulings_uri", "prints_search_uri", "collector_number", "digital", "flavor_text", "card_back_id", "artist", "artist_ids", "illustration_id", "border_color", "frame", "security_stamp", "full_art", "textless", "booster", "story_spotlight", "edhrec_rank", "preview", "related_uris", "prices", "tcgplayer_infinite_articles", "tcgplayer_infinite_decks", "edhrec", "purchase_uris", "cardmarket", "cardhoarder"]
    for keyword in keywords:
        if keyword in data:
            del data[keyword]
    
    clean_json = json.dumps(data)
    # convert into json dict & save:
    card1 = json.loads(clean_json)
    with open('./data/card1.json', 'w') as f:
        json.dump(card1, f)
    print(card1)


{'name': 'Urborg, Tomb of Yawgmoth', 'mana_cost': '', 'cmc': 0.0, 'type_line': 'Legendary Land', 'oracle_text': 'Each land is a Swamp in addition to its other land types.', 'colors': [], 'keywords': [], 'produced_mana': ['B']}


In [132]:
card2 = "Blood Moon"
response = requests.get(f'https://api.scryfall.com/cards/named?exact={card2} ', 
                 headers={'Accept': 'application/json'})
if response.status_code == 200:
    # Parse the JSON response into a Python dictionary to delete keywords
    data = response.json()
    print(card2)
    print(card2)
    keywords = ["object", "id", "oracle_id", "mtgo_foil_id", "rarity", "color_identity", "penny_rank", "multiverse_ids", "mtgo_id", "tcgplayer_id", "cardmarket_id", "lang", "released_at", "uri", "scryfall_uri", "layout", "frame_effects", "highres_image", "image_status", "image_uris", "legalities", "games", "reserved", "foil", "nonfoil", "finishes", "oversized", "promo", "reprint", "variation", "set_id", "set", "set_name", "set_type", "set_uri", "set_search_uri", "scryfall_set_uri", "rulings_uri", "prints_search_uri", "collector_number", "digital", "flavor_text", "card_back_id", "artist", "artist_ids", "illustration_id", "border_color", "frame", "security_stamp", "full_art", "textless", "booster", "story_spotlight", "edhrec_rank", "preview", "related_uris", "prices", "tcgplayer_infinite_articles", "tcgplayer_infinite_decks", "edhrec", "purchase_uris", "cardmarket", "cardhoarder"]
    for keyword in keywords:
        if keyword in data:
            del data[keyword]
    
    clean_json = json.dumps(data)
    # convert into json dict & save:
    card2 = json.loads(clean_json)
    with open('./data/card2.json', 'w') as f:
        json.dump(card2, f)
    print(card2)

Blood Moon
Blood Moon
{'name': 'Blood Moon', 'mana_cost': '{2}{R}', 'cmc': 3.0, 'type_line': 'Enchantment', 'oracle_text': 'Nonbasic lands are Mountains.', 'colors': ['R'], 'keywords': []}


In [133]:
# testing Prompt with input_variables as json
prompt_template = ChatPromptTemplate.from_template(
    template=prompt_continuous_effects,
    partial_variables={"card1": card1, "card2": card2},
)

- Use Chatmodel instead of LLM: more possible token in prompt
- Outputparser to parse message response from Chatmodel to str
- check testcases.md to compare expected outcome with ChatGPT's answer

In [134]:
chain = ({"rules_db": retriever_rules, "question": RunnablePassthrough()} | prompt_template | model | StrOutputParser())
answer = chain.invoke("What is the outcome of two played cards with continuous effects from Magic: the Gathering?")
print(answer)

1. Oracle texts:
- Urborg, Tomb of Yawgmoth: "Each land is a Swamp in addition to its other land types."
- Blood Moon: "Nonbasic lands are Mountains."

2. I would choose the Timestamp rule (613.1 to 613.7) for this case.

3. Quoting the Timestamp rule (613.1 to 613.7):
"Two effects are affecting the same creature: one from an Aura that says 'Enchanted creature has flying' and one from an Aura that says 'Enchanted creature loses flying.' Neither of these depends on the other, since nothing changes what they affect or what they’re doing to it. Applying them in timestamp order means the one that was generated last 'wins.'"

4. Explanation:
In this case, Urborg, Tomb of Yawgmoth and Blood Moon are both continuous effects that affect the type of lands in play. Since Urborg, Tomb of Yawgmoth was played first and Blood Moon was played second, according to the Timestamp rule, the effect of Blood Moon would override the effect of Urborg, Tomb of Yawgmoth.

5. Judging:
The continuous effect of B

- saving the answers as .txt for comparison by date & time: 

In [156]:
current_datetime = str(datetime.now().strftime("%Y-%m-%d %H-%M-%S"))
file_name = current_datetime+".txt"

with open(f'./data/answers/{file_name}', 'w') as f:
    f.write(answer)

In [22]:
# cleanup
rules_db.delete_collection()

## Case 2a: Opalescence vs. Humility
- timestamp rule determines the outcome: *Humility* is played last and it's effect overrides *Opalescence*

In [135]:
card1 = "Opalescence"
response = requests.get(f'https://api.scryfall.com/cards/named?exact={card1} ', 
                 headers={'Accept': 'application/json'})
if response.status_code == 200:
    # Parse the JSON response into a Python dictionary to delete keywords
    data = response.json()
    keywords = ["object", "id", "oracle_id", "mtgo_foil_id", "rarity", "color_identity", "penny_rank", "multiverse_ids", "mtgo_id", "tcgplayer_id", "cardmarket_id", "lang", "released_at", "uri", "scryfall_uri", "layout", "frame_effects", "highres_image", "image_status", "image_uris", "legalities", "games", "reserved", "foil", "nonfoil", "finishes", "oversized", "promo", "reprint", "variation", "set_id", "set", "set_name", "set_type", "set_uri", "set_search_uri", "scryfall_set_uri", "rulings_uri", "prints_search_uri", "collector_number", "digital", "flavor_text", "card_back_id", "artist", "artist_ids", "illustration_id", "border_color", "frame", "security_stamp", "full_art", "textless", "booster", "story_spotlight", "edhrec_rank", "preview", "related_uris", "prices", "tcgplayer_infinite_articles", "tcgplayer_infinite_decks", "edhrec", "purchase_uris", "cardmarket", "cardhoarder"]
    for keyword in keywords:
        if keyword in data:
            del data[keyword]
    
    clean_json = json.dumps(data)
    # convert into json dict & save:
    card1 = json.loads(clean_json)
    with open('./data/card1.json', 'w') as f:
        json.dump(card1, f)
    print(card1)

{'name': 'Opalescence', 'mana_cost': '{2}{W}{W}', 'cmc': 4.0, 'type_line': 'Enchantment', 'oracle_text': 'Each other non-Aura enchantment is a creature in addition to its other types and has base power and base toughness each equal to its mana value.', 'colors': ['W'], 'keywords': []}


In [136]:
card2 = "Humility"
response = requests.get(f'https://api.scryfall.com/cards/named?exact={card2} ', 
                 headers={'Accept': 'application/json'})
if response.status_code == 200:
    # Parse the JSON response into a Python dictionary to delete keywords
    data = response.json()
    print(card2)
    keywords = ["object", "id", "oracle_id", "mtgo_foil_id", "rarity", "color_identity", "penny_rank", "multiverse_ids", "mtgo_id", "tcgplayer_id", "cardmarket_id", "lang", "released_at", "uri", "scryfall_uri", "layout", "frame_effects", "highres_image", "image_status", "image_uris", "legalities", "games", "reserved", "foil", "nonfoil", "finishes", "oversized", "promo", "reprint", "variation", "set_id", "set", "set_name", "set_type", "set_uri", "set_search_uri", "scryfall_set_uri", "rulings_uri", "prints_search_uri", "collector_number", "digital", "flavor_text", "card_back_id", "artist", "artist_ids", "illustration_id", "border_color", "frame", "security_stamp", "full_art", "textless", "booster", "story_spotlight", "edhrec_rank", "preview", "related_uris", "prices", "tcgplayer_infinite_articles", "tcgplayer_infinite_decks", "edhrec", "purchase_uris", "cardmarket", "cardhoarder"]
    for keyword in keywords:
        if keyword in data:
            del data[keyword]
    
    clean_json = json.dumps(data)
    # convert into json dict & save:
    card2 = json.loads(clean_json)
    with open('./data/card2.json', 'w') as f:
        json.dump(card2, f)
    print(card2)

Humility
{'name': 'Humility', 'mana_cost': '{2}{W}{W}', 'cmc': 4.0, 'type_line': 'Enchantment', 'oracle_text': 'All creatures lose all abilities and have base power and toughness 1/1.', 'colors': ['W'], 'keywords': []}


In [137]:
prompt_template = ChatPromptTemplate.from_template(
    template=prompt_continuous_effects,
    partial_variables={"card1": card1, "card2": card2},
)

In [138]:
chain = ({"rules_db": retriever_rules, "question": RunnablePassthrough()} | prompt_template | model | StrOutputParser())
answer = chain.invoke("What is the outcome of two played cards with continuous effects from Magic: the Gathering?")
print(answer)

1. Oracle text of Opalescence: "Each other non-Aura enchantment is a creature in addition to its other types and has base power and base toughness each equal to its mana value."
Oracle text of Humility: "All creatures lose all abilities and have base power and toughness 1/1."

2. I choose to apply the Timestamp rule (613.1 to 613.7) in this case.

3. Quoting rule 613.9: "One continuous effect can override another. Sometimes the results of one effect determine whether another effect applies or what another effect does. Applying them in timestamp order means the one that was generated last 'wins.'"

4. I would apply the Timestamp rule in this case because it determines the order in which continuous effects are applied based on when they were generated. This ensures consistency and fairness in resolving conflicting effects.

5. Judging: In this scenario, the continuous effect of Humility would be applied in the game. Since Humility was played after Opalescence, its effect of making all cr

In [111]:
file_name = str(datetime.now().strftime("%Y-%m-%d %H-%M-%S"))
# file_name = current_datetime+".txt"

with open(f'./data/answers/case2_{file_name}.txt', 'w') as f:
    f.write(answer)

## Case 2b: Humility vs. Opalescence 
- because timestamp is important here, testing the case with changed order

In [139]:
card1 = "Humility"
response = requests.get(f'https://api.scryfall.com/cards/named?exact={card1} ', 
                 headers={'Accept': 'application/json'})
if response.status_code == 200:
    # Parse the JSON response into a Python dictionary to delete keywords
    data = response.json()
    keywords = ["object", "id", "oracle_id", "mtgo_foil_id", "rarity", "color_identity", "penny_rank", "multiverse_ids", "mtgo_id", "tcgplayer_id", "cardmarket_id", "lang", "released_at", "uri", "scryfall_uri", "layout", "frame_effects", "highres_image", "image_status", "image_uris", "legalities", "games", "reserved", "foil", "nonfoil", "finishes", "oversized", "promo", "reprint", "variation", "set_id", "set", "set_name", "set_type", "set_uri", "set_search_uri", "scryfall_set_uri", "rulings_uri", "prints_search_uri", "collector_number", "digital", "flavor_text", "card_back_id", "artist", "artist_ids", "illustration_id", "border_color", "frame", "security_stamp", "full_art", "textless", "booster", "story_spotlight", "edhrec_rank", "preview", "related_uris", "prices", "tcgplayer_infinite_articles", "tcgplayer_infinite_decks", "edhrec", "purchase_uris", "cardmarket", "cardhoarder"]
    for keyword in keywords:
        if keyword in data:
            del data[keyword]
    
    clean_json = json.dumps(data)
    # convert into json dict & save:
    card1 = json.loads(clean_json)
    with open('./data/card1.json', 'w') as f:
        json.dump(card1, f)
    print(card1)

{'name': 'Humility', 'mana_cost': '{2}{W}{W}', 'cmc': 4.0, 'type_line': 'Enchantment', 'oracle_text': 'All creatures lose all abilities and have base power and toughness 1/1.', 'colors': ['W'], 'keywords': []}


In [140]:
card2 = "Opalescence"
response = requests.get(f'https://api.scryfall.com/cards/named?exact={card2} ', 
                 headers={'Accept': 'application/json'})
if response.status_code == 200:
    # Parse the JSON response into a Python dictionary to delete keywords
    data = response.json()
    print(card2)
    keywords = ["object", "id", "oracle_id", "mtgo_foil_id", "rarity", "color_identity", "penny_rank", "multiverse_ids", "mtgo_id", "tcgplayer_id", "cardmarket_id", "lang", "released_at", "uri", "scryfall_uri", "layout", "frame_effects", "highres_image", "image_status", "image_uris", "legalities", "games", "reserved", "foil", "nonfoil", "finishes", "oversized", "promo", "reprint", "variation", "set_id", "set", "set_name", "set_type", "set_uri", "set_search_uri", "scryfall_set_uri", "rulings_uri", "prints_search_uri", "collector_number", "digital", "flavor_text", "card_back_id", "artist", "artist_ids", "illustration_id", "border_color", "frame", "security_stamp", "full_art", "textless", "booster", "story_spotlight", "edhrec_rank", "preview", "related_uris", "prices", "tcgplayer_infinite_articles", "tcgplayer_infinite_decks", "edhrec", "purchase_uris", "cardmarket", "cardhoarder"]
    for keyword in keywords:
        if keyword in data:
            del data[keyword]
    
    clean_json = json.dumps(data)
    # convert into json dict & save:
    card2 = json.loads(clean_json)
    with open('./data/card2.json', 'w') as f:
        json.dump(card2, f)
    print(card2)

Opalescence
{'name': 'Opalescence', 'mana_cost': '{2}{W}{W}', 'cmc': 4.0, 'type_line': 'Enchantment', 'oracle_text': 'Each other non-Aura enchantment is a creature in addition to its other types and has base power and base toughness each equal to its mana value.', 'colors': ['W'], 'keywords': []}


In [141]:
prompt_template = ChatPromptTemplate.from_template(
    template=prompt_continuous_effects,
    partial_variables={"card1": card1, "card2": card2},
)

In [142]:
chain = ({"rules_db": retriever_rules, "question": RunnablePassthrough()} | prompt_template | model | StrOutputParser())
answer = chain.invoke("What is the outcome of two played cards with continuous effects from Magic: the Gathering?")
print(answer)

1. Oracle texts:
- Humility: "All creatures lose all abilities and have base power and toughness 1/1."
- Opalescence: "Each other non-Aura enchantment is a creature in addition to its other types and has base power and base toughness each equal to its mana value."

2. I choose to apply the Timestamp rule (613.1 to 613.7) in this case.

3. Quoting the rule:
- From Document(page_content='613.9. One continuous effect can override another. Sometimes the results of one effect determine whether another effect applies or what another effect does. Example: Two effects are affecting the same creature: one from an Aura that says “Enchanted creature has flying” and one from an Aura that says “Enchanted creature loses flying.” Neither of these depends on the other, since nothing changes what they affect or what they’re doing to it. Applying them in timestamp order means the one that was generated last “wins.” The same process would be followed, and the same result reached, if either of the effects

In [143]:
file_name = str(datetime.now().strftime("%Y-%m-%d %H-%M-%S"))
# file_name = current_datetime+".txt"

with open(f'./data/answers/case2_{file_name}.txt', 'w') as f:
    f.write(answer)

In [144]:
# cleanup
rules_db.delete_collection()